In [ ]:
!pip install bitsandbytes # we need latest transformers for this
!pip install --upgrade accelerate
!pip install --upgrade git+https://github.com/huggingface/transformers
!pip install --upgrade torch torchvision

!pip install datasets==2.16.0
!pip install git+https://github.com/huggingface/peft.git@8dd45b75d7eabe7ee94ecb6a19d552f2aa5e98c6  # specific commit that doesn't break!
!pip install wandb
!pip install scipy

In [ ]:
# Optional WandB setup
import wandb
wandb.init()

In [ ]:
import torch
from datetime import datetime
import os
import sys

from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq, BitsAndBytesConfig
from datasets import load_dataset
import torch,einops
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
dataset = load_dataset("json",data_files="./dataset_construct/datasets/combined_dataset.json",split="train") # change this to your dataset path
print(dataset[1000])

In [ ]:
train_dataset = dataset.train_test_split(test_size=0.1)["train"]
eval_dataset = dataset.train_test_split(test_size=0.1)["test"]

In [ ]:
base_model_name ="./CodeLlama-13b-Python-hf" # change this to your model path
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
tokenizer = AutoTokenizer.from_pretrained("./CodeLlama-13b-Python-hf") # change this to your model path

In [ ]:
device_map = {"": 0}

In [ ]:
tokenizer.add_eos_token = True
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"

In [ ]:
def tokenize(sample):
    prompt = sample['text']
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=256,
        padding=False,
        return_tensors=None,
    )

    # "self-supervised learning" means the labels are also the inputs:
    result["labels"] = result["input_ids"].copy()

    return result

tokenized_train_dataset = train_dataset.map(tokenize)
tokenized_val_dataset = eval_dataset.map(tokenize)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True,
    use_auth_token=True
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

In [ ]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
output_dir = "./results"
batch_size = 128
per_device_train_batch_size = 4
gradient_accumulation_steps = batch_size // per_device_train_batch_size

training_args = TrainingArguments(
        per_device_train_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        num_train_epochs=1,
        warmup_steps=100,
        max_steps=500,
        learning_rate=3e-4,
        fp16=True,
        logging_steps=10,
        optim="adamw_torch",
        evaluation_strategy="steps", # if val_set_size > 0 else "no",
        save_strategy="steps",
        eval_steps=20,
        save_steps=100,
        output_dir=output_dir,
        # save_total_limit=3,
        load_best_model_at_end=False,
        # ddp_find_unused_parameters=False if ddp else None,
        group_by_length=True, # group sequences of roughly the same length together to speed up training
        report_to="wandb", # if use_wandb else "none",
        run_name=f"codellama-{datetime.now().strftime('%Y-%m-%d-%H-%M')}", # if use_wandb else None,
    )
max_seq_length = 512

trainer = SFTTrainer(
    model=base_model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_args,
)

In [ ]:
trainer.train()

In [ ]:
import os
output_dir = os.path.join(output_dir, "final_checkpoint")
trainer.model.save_pretrained(output_dir)